In [ ]:
from utils.binaries import *

In [ ]:
runlist =  pd.read_csv(f'/cr/data01/filip/xy-calibration/config/calib_runlists/calib_runs_2022-10.list', sep=';')
runlist['comment'] = [comment.strip() for comment in runlist['comment']]
runlist['source'] = [source.strip() for source in runlist['source']]

# add new runlists here!
for month in ['2023-10', '2023-11']:
    this_runlist = pd.read_csv(f'/cr/data01/filip/xy-calibration/config/calib_runlists/calib_runs_{month}.list', sep=';')
    this_runlist['comment'] = [comment.strip() for comment in this_runlist['comment']]
    this_runlist['source'] = [source.strip() for source in this_runlist['source']]

    runlist = pd.concat([runlist, this_runlist], ignore_index=True)



In [ ]:
runlist

In [ ]:
only_xy_runs = runlist
only_xy_runs = runlist.drop(runlist[runlist['comment'] != '""'].index)
only_xy_runs = only_xy_runs.drop(only_xy_runs[only_xy_runs['source'] != '"OLO"'].index)
only_xy_runs = only_xy_runs.drop(only_xy_runs[only_xy_runs['forDB'] == 0].index)
only_xy_runs

In [ ]:
XY_measurements = {}

for index, measurement in only_xy_runs.iterrows():

    try:
        _ = XY_measurements[measurement['telescope'].strip()]
    except KeyError:
        XY_measurements[measurement['telescope'].strip()] = []

    (runid, telescope), date = measurement[:2], measurement['date'].strip()

    same_day = runlist[[d.strip() == date for d in runlist['date']]]
    same_telescope = same_day[[t.strip() == telescope.strip() for t in same_day['telescope']]]

    year, month, day = date.split('-')

    XY_measurement = {'XY' : measurement['#runid'],
                      'CalA_closed_shutter' : [None, None],
                      'CalA_open_shutter' : [None, None], 
                      'date' : f"{year}{month}".replace('10','oct').replace('11','nov')}

    # find Cal A closed shutter
    for candidate_index, candidate in same_telescope.iterrows():
        if candidate['comment'] == '"Cal A"':
            if candidate_index < index: XY_measurement['CalA_closed_shutter'][0] = candidate['#runid']
            if candidate_index > index: XY_measurement['CalA_closed_shutter'][1] = candidate['#runid']
        if candidate['comment'] == '"Cal A open shutter"':
            if candidate_index < index: XY_measurement['CalA_open_shutter'][0] = candidate['#runid']
            if candidate_index > index: XY_measurement['CalA_open_shutter'][1] = candidate['#runid']

    XY_measurements[telescope.strip()].append(XY_measurement)

In [ ]:
XY_measurements

In [ ]:
import pickle

# save events to disk
with open('/cr/users/filip/bin/utils/Auger/FD/xy_measurements.pkl', "wb") as f:
    pickle.dump(XY_measurements, f, pickle.HIGHEST_PROTOCOL)

In [1]:
from utils.Auger.FD import XYscanner

20:11:58 (   +1.8s) [INFO   ] -- import logging
20:11:59 (  +397ms) [INFO   ] -- import matplotlib.pyplot as plt
20:11:59 (  +599ms) [INFO   ] -- import seaborn as so
20:11:59 (    +7ms) [DEBUG  ] -- font size set to 9.5
20:11:59 (    +0ms) [DEBUG  ] -- label size set to 13.0
20:11:59 (    +1ms) [DEBUG  ] -- figure size set to [6.6, 3.3]
20:11:59 (    +1ms) [DEBUG  ] -- markersize set to 2.0
20:11:59 (    +1ms) [DEBUG  ] -- usetex set to False
20:11:59 (    +2ms) [INFO   ] -- import numpy as np
20:11:59 (    +7ms) [INFO   ] -- import uncertainties
20:11:59 (    +1ms) [INFO   ] -- import pandas as pd
20:11:59 (    +1ms) [WARNING] -- pandas support to be dropped at some point!
20:11:59 (    +2ms) [INFO   ] -- import binaries.tools as tools
20:11:59 (    +1ms) [INFO   ] -- import plotting.tools as plot


In [6]:
print(XYscanner.get_run_numbers('co3', '2023nov'))

{'XY': '15941x', 'CalA_closed_shutter': ['15929m21', '15956m21'], 'CalA_open_shutter': ['15940', '15950'], 'date': '2023nov'}


In [4]:
import pickle

with open('/cr/users/filip/bin/utils/Auger/FD/xy_measurements.pkl', 'rb') as f:
    data = pickle.load(f)


print(data['co3'])

[{'XY': '15448', 'CalA_closed_shutter': ['15440m21', '15459m21'], 'CalA_open_shutter': ['15447', '15449'], 'date': '2022oct'}, {'XY': '15941x', 'CalA_closed_shutter': ['15929m21', '15956m21'], 'CalA_open_shutter': ['15940', '15950'], 'date': '2023nov'}]


In [5]:
for measurement in data['co3']:
    print(measurement)

{'XY': '15448', 'CalA_closed_shutter': ['15440m21', '15459m21'], 'CalA_open_shutter': ['15447', '15449'], 'date': '2022oct'}
{'XY': '15941x', 'CalA_closed_shutter': ['15929m21', '15956m21'], 'CalA_open_shutter': ['15940', '15950'], 'date': '2023nov'}
